# Text Wrangling and Regex

Working with text: applying string methods and regular expressions

In [34]:
from google.colab import drive
drive.mount('/content/drive')
from datetime import datetime

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import zipfile
import pandas as pd

## Demo 1: Canonicalizing County Names

Load both **county_and_state.csv** and **county_and_population.csv**

In [36]:
# display both frames
#code Here
df1 = pd.read_csv('/content/drive/My Drive/county_and_population.csv')
df1

,County,Population
0,DeWitt,16798
1,Lac Qui Parle,8067
2,Lewis & Clark,55716
3,St. John the Baptist,43044


In [37]:
df2 = pd.read_csv('/content/drive/My Drive/county_and_state.csv')
df2

,County,State
0,De Witt County,IL
1,Lac qui Parle County,MN
2,Lewis and Clark County,MT
3,St John the Baptist Parish,LS


Both of these DataFrames share a "County" column. Unfortunately, formatting differences mean that we can't directly merge the two DataFrames using the "County"s.

In [38]:
#merge both frames usding pd.merge
#The output would only show the header. We will resolve it later on.
merge_data = pd.merge(df1, df2, on='County', how='inner')
merge_data

,County,Population,State


To address this, we can **canonicalize** the "County" string data to apply a common formatting.

In [39]:
def canonicalize_county(county_series):   #take help from your slides to cover this part
  return (county_series
                  # lowercase
                .str.lower()
                  # remove space
                .str.replace(' ', '')
                  # replace &
                .str.replace('&', 'and')
                  # remove dot
                .str.replace('.', '')
                  # remove "county"
                .str.replace('county', '')
                  # remove "parish"
                .str.replace('parish', ''))

Apply canonicalize_county on 'County' columns in both frames.

In [40]:
# Code Here
df1['County'] = canonicalize_county(df1['County'])
df2['County'] = canonicalize_county(df2['County'])

Now, the merge works as expected!

In [41]:
# code Here
#now merge df1 and df2
merged_data = pd.merge(df1, df2, on='County', how='inner')
merged_data

,County,Population,State
0,dewitt,16798,IL
1,lacquiparle,8067,MN
2,lewisandclark,55716,MT
3,stjohnthebaptist,43044,LS


## Demo 2: Extracting Log Data

Load log.txt

In [42]:
#code Here
        # Process each line as needed
#Hint: the file will be read almost the same way as we read the json file.
#Use the file.readlines function to read the lines in the txt file
file_path = '/content/drive/My Drive/log.txt'
with open(file_path, 'r') as file:
  data = file.readlines()
data

['169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"\n',
 '193.205.203.3 - - [2/Feb/2005:17:23:6 -0800] "GET /stat141/Notes/dim.html HTTP/1.0" 404 302 "http://eeyore.ucdavis.edu/stat141/Notes/session.html"\n',
 '169.237.46.240 - "" [3/Feb/2006:10:18:37 -0800] "GET /stat141/homework/Solutions/hw1Sol.pdf HTTP/1.1"\n']

Suppose we want to extract the day, month, year, hour, minutes, seconds, and timezone. Looking at the data, we see that these items are not in a fixed position relative to the beginning of the string. That is, slicing by some fixed offset isn't going to work.

Instead, we'll need to use some more sophisticated thinking. Let's focus on only the first line of the file.

In [43]:
# code here
data[0]

'169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"\n'

Apply string functions of python to extract date from first entry in log file.

In [44]:
# you might take help from the slides
                                        # find the text enclosed in square brackets
                                        # split up the date/month/year
                                        # split up the hour:minute:second
                                        # split the timezone after the blank space
first_line = data[0]
full_time = first_line.split('[')[1].split(']')[0]
day,month,rest = full_time.split('/')
year,hour,minutes,rest = rest.split(':')
seconds,time_zone = rest.split(' ')
result = (day, month, year, hour, minutes, seconds, time_zone)
print(result)

('26', 'Jan', '2014', '10', '47', '58', '-0800')


This worked, but felt fairly "hacky" – the code above isn't particularly elegant. A much more sophisticated but common approach is to extract the information we need using a *regular expression*.


# Regular Expressions


## String Extraction with Regex

Python `re.findall` returns a list of all extracted matches: extract numbers from given string.

In [45]:
import re

text = "My social security number is 123-45-6789 bro, or actually maybe it’s 321-45-6789.";
# code Here
ssn_pattern = r"[0-9]{3}-\d{2}-\d{4}"
ssns = re.findall(ssn_pattern, text)
print(ssns)
#Expected output:  ['123-45-6789', '321-45-6789']

['123-45-6789', '321-45-6789']


<br/>

Now, let's see vectorized extraction in `pandas`:

 `.str.findall` returns a `Series` of lists of all matches in each record.

In [46]:
data = ['987-65-4321','forty','123-45-6789 bro or 321-45-6789','999-99-9999']
# code here to convert the above data into a dataframe
data = pd.DataFrame(data)
data

,0
0,987-65-4321
1,forty
2,123-45-6789 bro or 321-45-6789
3,999-99-9999


Find all entries containing numbers in dataframe.

In [47]:
# -> Series of lists
# code Here
#pattern = r"[0-9]{3}-[0-9]{2}-[0-9]{4}"
pattern = r"(\d+)-(\d+)-(\d+)"
df_ssn = data[0].str.findall(pattern)
df_ssn
#Expected Output:
#  0                   [987, 65, 4321]
#  1                                []
#  2    [123, 45, 6789, 321, 45, 6789]
#  3                   [999, 99, 9999]
#  Name: SSN, dtype: object

,0
0,"[(987, 65, 4321)]"
1,[]
2,"[(123, 45, 6789), (321, 45, 6789)]"
3,"[(999, 99, 9999)]"


## Extraction Using Regex Capture Groups

The Python function `re.findall`, in combination with parentheses returns specific substrings (i.e., **capture groups**) within each matched string, or **match**.

In [48]:
text = """I will meet you at 08:30:00 pm tomorrow"""
# code here
pattern = r".*(\d\d):(\d\d):(\d\d).*"
matches = re.findall(pattern, text)
matches

[('08', '30', '00')]

In [49]:
# the three capture groups in the first matched string
hour, minute, second = matches[0]
hour

'08'

<br/>

In `pandas`, we can use `.str.extract` to extract each capture group of **only the first match** of each record into separate columns.

In [50]:
# back to SSNs
data_ssn = pd.DataFrame(data)
data_ssn

,0
0,987-65-4321
1,forty
2,123-45-6789 bro or 321-45-6789
3,999-99-9999


In [51]:
# Will extract the first match of all groups
# code here
pattern = r"(\d+-\d+-\d+)"
extracted_data1 = data_ssn[0].str.extract(pattern)
extracted_data1

,0
0,987-65-4321
1,NaN
2,123-45-6789
3,999-99-9999


Alternatively, `.str.extractall` extracts **all matches** of each record into separate columns. Rows are then MultiIndexed by original record index and match index.

In [52]:
# -> DataFrame, one row per match
# code Here
extracted_data2 = data_ssn[0].str.extractall(pattern)
extracted_data2

0
  match             
0 0      987-65-4321
2 0      123-45-6789
  1      321-45-6789
3 0      999-99-9999

## Canonicalization with Regex

In regular Python, canonicalize with `re.sub` (standing for "substitute"):

In [53]:
text = '<div><td valign="top">Moo</td></div>'

#code here to extract the word Moo
pattern = r"<[^>]+>"
sub_data = re.sub(pattern, '', text)
sub_data

'Moo'

<br/>

In `pandas`, canonicalize with `Series.str.replace`.

In [54]:
# example dataframe of strings, convert in dataframe
df_html = ['<div><td valign="top">Moo</td></div>',
                   '<a href="http://ds100.org">Link</a>',
                   '<b>Bold text</b>']
df_html

['<div><td valign="top">Moo</td></div>',
 '<a href="http://ds100.org">Link</a>',
 '<b>Bold text</b>']

In [55]:
# Series -> Series
#Extract only words from the above given df_html like Moo, Link and Bold text
pattern = r"<[^>]+>"
expected_values = []
for values in df_html:
  expected_values.append(re.sub(pattern, '', values))
expected_values


['Moo', 'Link', 'Bold text']


# Revisiting Text Log Processing using Regex

### Python `re` version

In [56]:
file_path = '/content/drive/My Drive/log.txt'
with open(file_path, 'r') as file:
  log_lines = file.readlines()
line = log_lines
display(line)
# code here

['169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"\n',
 '193.205.203.3 - - [2/Feb/2005:17:23:6 -0800] "GET /stat141/Notes/dim.html HTTP/1.0" 404 302 "http://eeyore.ucdavis.edu/stat141/Notes/session.html"\n',
 '169.237.46.240 - "" [3/Feb/2006:10:18:37 -0800] "GET /stat141/homework/Solutions/hw1Sol.pdf HTTP/1.1"\n']

### `pandas` version

In [57]:
# code here
time_stamp=[]
for i in line:
  time_stamp.append(i.split('[')[1].split(']')[0])
time_stamp

['26/Jan/2014:10:47:58 -0800',
 '2/Feb/2005:17:23:6 -0800',
 '3/Feb/2006:10:18:37 -0800']

In [88]:
days = []
months = []
years = []
hours = []
minutes = []
seconds = []
time_zones = []

for ts in time_stamp:
    # Extract day, month, and the rest (year and time)
    day, month, rest = ts.split('/')
    year, hour, minutes_part, rest = rest.split(':')
    seconds_part, time_zone = rest.split(' ')

    # Append each component to the respective list
    days.append(day)
    months.append(month)
    years.append(year)
    hours.append(hour)
    minutes.append(minutes_part)
    seconds.append(seconds_part)
    time_zones.append(time_zone)

#print("Days:", days)
#print("Months:", months)
#print("Years:", years)
#print("Hours:", hours)
#print("Minutes:", minutes)
#print("Seconds:", seconds)
#print("Time Zones:", time_zones)

data = {
    'Day': days,
    'Month': months,
    'Year': years,
    'Hour': hours,
    'Minute': minutes,
    'Second': seconds,
    'Time Zone': time_zones
}

df = pd.DataFrame(data)

# Output the DataFrame
df

,Day,Month,Year,Hour,Minute,Second,Time Zone
0,26,Jan,2014,10,47,58,-0800
1,2,Feb,2005,17,23,6,-0800
2,3,Feb,2006,10,18,37,-0800


Option 1: `Series.str.findall`

In [74]:
# code here
pattern = r"(\d{1,2})/(\w{3})/(\d{4}):(\d{1,2}):(\d{1,2}):(\d{1,2}) ([-]\d{4})"
all_matches = []
for ts in time_stamp:
    matches = re.findall(pattern, ts)
    all_matches.extend(matches)
all_matches
#for match in all_matches:
 #   print(match)

[('26', 'Jan', '2014', '10', '47', '58', '-0800'),
 ('2', 'Feb', '2005', '17', '23', '6', '-0800'),
 ('3', 'Feb', '2006', '10', '18', '37', '-0800')]

<br/>

Option 2: `Series.str.extractall`

In [84]:
# code here
df_new = pd.DataFrame(time_stamp, columns=['timestamps'])
pattern = r"(\d{1,2})/(\w{3})/(\d{4}):(\d{1,2}):(\d{1,2}):(\d{1,2}) ([-]\d{4})"
matches = df_new['timestamps'].str.extractall(pattern)
matches.reset_index(drop=True, inplace=True)
matches

,0,1,2,3,4,5,6
0,26,Jan,2014,10,47,58,-0800
1,2,Feb,2005,17,23,6,-0800
2,3,Feb,2006,10,18,37,-0800


Wrangling either of these two DataFrames into a nice format (like below) is left as an exercise for you!


||Day|Month|Year|Hour|Minute|Second|Time Zone|
|---|---|---|---|---|---|---|---|
|0|26|Jan|2014|10|47|58|-0800|
|1|2|Feb|2005|17|23|6|-0800|
|2|3|Feb|2006|10|18|37|-0800|


In [85]:
# your code here
mt_df = matches.rename(columns={0:'Day', 1:'Month', 2:'Year', 3:'Hour', 4:'Minute', 5:'Second', 6:'Time Zone'})
mt_df

,Day,Month,Year,Hour,Minute,Second,Time Zone
0,26,Jan,2014,10,47,58,-0800
1,2,Feb,2005,17,23,6,-0800
2,3,Feb,2006,10,18,37,-0800
